In [1]:
using ModelingToolkit, IntervalSets, Sophon, CairoMakie, Lux
using Optimization, OptimizationOptimJL

@parameters x,t
@variables θ(..), w(..), α(..)
Dₜ = Differential(t)
Dₜ² = Differential(t)^2
Dₓ = Differential(x)
Dₓ² = Differential(x)^2

g(x,t) = cos(t) - π/2 * sin(x)*cos(t)

eqs = [
    α(x,t) * Dₜ²(θ(x,t)) - Dₓ²(θ(x,t)) + θ(x,t) - Dₓ(w(x,t)) ~ 0.0,
    Dₜ²(w(x,t)) - Dₓ²(w(x,t)) + Dₓ(θ(x,t)) ~ g(x,t)]

bcs = [
    θ(x, 0.0) ~ π/2*cos(x) + x - π/2,
    Dₜ(θ(x, 0.0)) ~ 0.0,
    w(x, 0.0) ~ π/2*sin(x),
    Dₜ(w(x, 0.0)) ~ 0.0,
    θ(0.0, t) ~ 0.0,
    θ(π, t) ~ 0.0,
    w(0.0, t) ~ 0.0, 
    w(π, t) ~ 0.0]

    
domains = [x ∈ Interval(0.0, π),
           t ∈ Interval(0.0, 1.0)]

@named timo = PDESystem(eqs, bcs, domains, [x,t], [θ(x,t),w(x,t),α(x,t)])

PDESystem
Equations: Equation[Differential(t)(Differential(t)(θ(x, t)))*α(x, t) + θ(x, t) - Differential(x)(Differential(x)(θ(x, t))) - Differential(x)(w(x, t)) ~ 0.0, Differential(t)(Differential(t)(w(x, t))) + Differential(x)(θ(x, t)) - Differential(x)(Differential(x)(w(x, t))) ~ cos(t) - 1.5707963267948966cos(t)*sin(x)]
Boundary Conditions: Equation[θ(x, 0.0) ~ x + 1.5707963267948966cos(x) - 1.5707963267948966, Differential(t)(θ(x, 0.0)) ~ 0.0, w(x, 0.0) ~ 1.5707963267948966sin(x), Differential(t)(w(x, 0.0)) ~ 0.0, θ(0.0, t) ~ 0.0, θ(π, t) ~ 0.0, w(0.0, t) ~ 0.0, w(π, t) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(x, 0.0..3.141592653589793), Symbolics.VarDomainPairing(t, 0.0..1.0)]
Dependent Variables: Num[θ(x, t), w(x, t), α(x, t)]
Independent Variables: Num[x, t]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [2]:
pinn = PINN(θ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            w = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            α = ConstantFunction())

sampler = QuasiRandomSampler(2000, 500)
strategy = NonAdaptiveTraining(1, 10)

NonAdaptiveTraining{Int64, Int64}(1, 10)

In [15]:
# generate training data
x_train = [0.2,0.8,1.8,2.6,3.0]
t_train = 0.0:0.001:1.0
θ_analytic(x,t) = (π/2*cos(x) + x - π/2)*cos(t)
w_analytic(x,t) = π/2*sin(x)*cos(t)

coord_data = stack(vec(vcat.(x_train, t_train')))
θ_data = reshape(θ_analytic.(x_train, t_train'), 1, :)
w_data = reshape(w_analytic.(x_train, t_train'), 1, :)

1×5005 Matrix{Float64}:
 0.312069  1.12682  1.52972  0.809748  …  0.826509  0.437509  0.119769

In [16]:
function additional_loss(phi, ps)
    (sum(abs2, phi.θ(coord_data, ps.θ) .- θ_data) + sum(abs2, phi.w(coord_data, ps.w) .- w_data))/50000
end

additional_loss (generic function with 1 method)